In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle
from langchain import hub


# Document loader

In [2]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())

for page in pages:
    page.page_content = page.page_content.replace("Imprimerie Officielle de la République Tunisienne", "")


# Splitter


In [5]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)


28127

# Vectorstores
## for now BAAI/bge-m3 is the best embedder

#### In the cloud (pinecone)

In [6]:
import os
os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
# embedder = HuggingFaceEmbeddings(
#     model_name = 'dangvantuan/sentence-camembert-large'
# )

index_name = "pfa"

#docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)
#docsearch = PineconeVectorStore(index_name=index_name,embedding=embedder)

.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

#### Locally (chroma)

In [42]:

embedder = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3"
)
persist_directory = 'docs/chroma_bge/'
#vectordb.delete_collection()
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedder,
    persist_directory=persist_directory
) 
vectordb.persist()
print(vectordb._collection.count())

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

28127


# Retrieval
### 3 methodes

In [49]:

query="quels sont mes droits en tant qu'un enfant?"

#méthode 1 avec retriever
# retriever = vectordb.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

#méthode 2 avec mmr
found_docs = vectordb.max_marginal_relevance_search(query, k=3, fetch_k=100)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = vectordb.similarity_search_with_score(query, k=5)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)
    

1. Article 5.- Chaque enfant a droit à une identité dès sa naissance. 
L'identité est constituée du prénom, du nom de famille, de la date 
de naissance et de la nationalité. 
Article 6.- Chaque enfant a droit au respect de sa vie privée, tout 
en considérant les droits et les responsabilités de ses parents ou de ceux qui en ont la charge, conformément à la loi. 


2. civile solidaire, fondée sur l'indissociabilité entre la conscience des droits et le respect des devoirs, où prévalent les valeurs de l'équité, de 
la tolérance et de la modération. 
4/ Inscrire les droits de l'enfant à la sauvegarde et à la protection 
dans le contexte des grandes options nationales, qui ont fait des droits de l'homme de nobles idéaux qui orientent la volonté du Tunisien et lui permettent de développer sa réalité et d'accéder à un meilleur vécu, 


3. Article 70.- La constitution de la partie civile n'est pas admise 
devant les juridictions pour enfants, ceci n'empêche pas de recourir à la médiation, conf

# Prompt Engineering and generation

In [53]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")
prompt = prompt.invoke({"context":format_docs(found_docs), "question":query})
print(prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: quels sont mes droits en tant qu'un enfant? \nContext: Article 5.- Chaque enfant a droit à une identité dès sa naissance. \nL'identité est constituée du prénom, du nom de famille, de la date \nde naissance et de la nationalité. \nArticle 6.- Chaque enfant a droit au respect de sa vie privée, tout \nen considérant les droits et les responsabilités de ses parents ou de ceux qui en ont la charge, conformément à la loi.\n\ncivile solidaire, fondée sur l'indissociabilité entre la conscience des droits et le respect des devoirs, où prévalent les valeurs de l'équité, de \nla tolérance et de la modération. \n4/ Inscrire les droits de l'enfant à la sauvegarde et à la protection \ndans le contexte des grandes options na

In [54]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke(prompt)

'As an assistant for question-answering tasks, I can provide information on the rights of children based on the provided context. According to Article 5 of the context, every child has the right to an identity from birth, which includes their given name, family name, date of birth, and nationality. Additionally, Article 6 states that every child has the right to respect for their private life, while considering the rights and responsibilities of their parents or those who have their care, as per the law.\n\nFurthermore, Article 70 and 71 of the context provide information on the legal protection of children. Children aged between 13 to 18 years old who commit a qualified offense, contravention, felony, or crime are not tried in front of common law penal jurisdictions. Instead, they are tried by the juvenile judge or the juvenile court.\n\nIn summary, children have the right to an identity, respect for their private life, and protection from legal proceedings as minors.'